In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=abm_14_2_0_reporting;'
                    'Trusted_Connection=yes;')

query ='''-- connect to ddamwsql2016
SELECT [scenario_id]
      ,[measure]
      ,[metric]
      ,ROUND([value]*100,1) as [rounded_val] -- change this depending on value, i.e. if percent or other value
      ,[value] as [og_val]
      ,[updated_by]
      ,[updated_date]
  FROM [abm_14_2_0_reporting].[rp_2021].[results]
  WHERE scenario_id = 767
  AND measure like 'M-1%'
  --AND metric like 'Population Access%' -- change this depending on the measure'''

sql_data =  pd.read_sql_query(query, conn)

In [6]:
lst = list(sql_data['metric'])

lst = [s.split('-') for s in lst]

#print(lst)

[['Low Income Access Pct ', ' Bike'], ['Low Income Access Pct ', ' Bike + Walk + MM + MT'], ['Low Income Access Pct ', ' Drive Alone'], ['Low Income Access Pct ', ' Transit ', ' Speed One'], ['Low Income Access Pct ', ' Walk'], ['Low Income Access Pct ', ' Walk + MM + MT'], ['Minority Access Pct ', ' Bike'], ['Minority Access Pct ', ' Bike + Walk + MM + MT'], ['Minority Access Pct ', ' Drive Alone'], ['Minority Access Pct ', ' Transit ', ' Speed One'], ['Minority Access Pct ', ' Walk'], ['Minority Access Pct ', ' Walk + MM + MT'], ['Non', 'Low Income Access Pct ', ' Bike'], ['Non', 'Low Income Access Pct ', ' Bike + Walk + MM + MT'], ['Non', 'Low Income Access Pct ', ' Drive Alone'], ['Non', 'Low Income Access Pct ', ' Transit ', ' Speed One'], ['Non', 'Low Income Access Pct ', ' Walk'], ['Non', 'Low Income Access Pct ', ' Walk + MM + MT'], ['Non', 'Minority Access Pct ', ' Bike'], ['Non', 'Minority Access Pct ', ' Bike + Walk + MM + MT'], ['Non', 'Minority Access Pct ', ' Drive Alone'

In [11]:
first_values = [inner_lst[0] if inner_lst[0] != 'Non' else inner_lst[0]+'-'+inner_lst[1] for inner_lst in lst]
second_values = [inner_lst[1] if inner_lst[0] != 'Non' else inner_lst[2] for inner_lst in lst]

In [12]:
sql_data['population'] = first_values
sql_data['mode'] = second_values

In [13]:
sql_data

,scenario_id,measure,metric,rounded_val,og_val,updated_by,updated_date,population,mode
0,767,M-1-a,Low Income Access Pct - Bike,96.0,0.959586,SANDAGNET\ndh,2022-12-14 12:49:00,Low Income Access Pct,Bike
1,767,M-1-a,Low Income Access Pct - Bike + Walk + MM + MT,96.0,0.959586,SANDAGNET\ndh,2022-12-14 12:50:00,Low Income Access Pct,Bike + Walk + MM + MT
2,767,M-1-a,Low Income Access Pct - Drive Alone,98.5,0.984596,SANDAGNET\ndh,2022-12-14 12:51:00,Low Income Access Pct,Drive Alone
3,767,M-1-a,Low Income Access Pct - Transit - Speed One,70.4,0.703995,SANDAGNET\ndh,2022-12-14 12:51:00,Low Income Access Pct,Transit
4,767,M-1-a,Low Income Access Pct - Walk,76.6,0.765794,SANDAGNET\ndh,2022-12-14 12:51:00,Low Income Access Pct,Walk
...,...,...,...,...,...,...,...,...,...
191,767,M-1-c - Mobility Hubs,Non-Senior Access Pct - Transit - Speed One,95.5,0.954789,SANDAGNET\ndh,2022-12-14 13:02:00,Non-Senior Access Pct,Transit
192,767,M-1-c - Mobility Hubs,Population Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 13:02:00,Population Access Pct,Drive Alone
193,767,M-1-c - Mobility Hubs,Population Access Pct - Transit - Speed One,95.5,0.955271,SANDAGNET\ndh,2022-12-14 13:02:00,Population Access Pct,Transit
194,767,M-1-c - Mobility Hubs,Senior Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 13:02:00,Senior Access Pct,Drive Alone


In [16]:
sql_data['mode'] = sql_data['mode'].replace({' Bike':'Bike', 
                                            ' Bike + Walk + MM + MT':'Walk, Bike, Micromobility, Microtransit',
                                            ' Drive Alone':'Driving (drive alone)',
                                            ' Transit ': 'Transit - Accessed by Walk and or Flexible Fleet - Speed One',
                                            ' Walk':'Walk',
                                            ' Walk + MM + MT':'Walk, Micromobility, Microtransit'
                                            })

In [44]:
output_rank= {'Bike':2, 
                                            'Walk, Bike, Micromobility, Microtransit':4,
                                            'Driving (drive alone)':6,
                                            'Transit - Accessed by Walk and or Flexible Fleet - Speed One':5,
                                            'Walk':1,
                                            'Walk, Micromobility, Microtransit':3
                                            }

In [46]:
sql_data['output_ranking'] = [output_rank[mode] for mode in sql_data['mode']]

In [49]:
final_output = sql_data.sort_values('output_ranking')

In [52]:
set(final_output['measure'])

{'M-1-a',
 'M-1-a - Mobility Hubs',
 'M-1-b',
 'M-1-b - Mobility Hubs',
 'M-1-c',
 'M-1-c - Mobility Hubs'}

In [56]:
set(final_output['population'])

{'Low Income Access Pct ',
 'Minority Access Pct ',
 'Non-Low Income Access Pct ',
 'Non-Minority Access Pct ',
 'Non-Senior Access Pct ',
 'Population Access Pct ',
 'Senior Access Pct '}

In [62]:
set(df['scenario_id'])

{767}

In [90]:
# measures = ['M-1-a',
#  'M-1-a - Mobility Hubs',
#  'M-1-b',
#  'M-1-b - Mobility Hubs',
#  'M-1-c',
#  'M-1-c - Mobility Hubs']

measures = ['M-1-a','M-1-b','M-1-c']



# pops = ['Low Income Access Pct ',
#  'Minority Access Pct ',
#  'Non-Low Income Access Pct ',
#  'Non-Minority Access Pct ',
#  'Non-Senior Access Pct ',
#  'Population Access Pct ',
#  'Senior Access Pct ']

pops = ['Low Income Access Pct ','Non-Low Income Access Pct ','Minority Access Pct ','Non-Minority Access Pct ','Senior Access Pct ','Non-Senior Access Pct ']

In [91]:
for measure in measures:
    for ind_pop in pops:
        print(measure + ' - ' + ind_pop + ' -  Regional')
        df = final_output[final_output['measure'] == measure]
        df = df[df['population'] == ind_pop]
        print(df[['mode', 'rounded_val']])

        print(measure + ' - ' + ind_pop + ' -  Mohub')
        df = final_output[final_output['measure'] == measure+' - Mobility Hubs']
        df = df[df['population'] == ind_pop]
        print(df[['mode', 'rounded_val']])

        # print(measure)
        # print(measure+' - Mobility Hubs')
        # print(ind_pop)



        # print(measure + ' - ' + ind_pop)
        # df = final_output[final_output['measure'] == measure]
        # df = df[df['population'] == ind_pop]
        # print(df[['mode', 'rounded_val']])

M-1-a - Low Income Access Pct  -  Regional
                                                mode  rounded_val
4                                               Walk         76.6
0                                               Bike         96.0
5                  Walk, Micromobility, Microtransit         77.1
1            Walk, Bike, Micromobility, Microtransit         96.0
3  Transit - Accessed by Walk and or Flexible Fle...         70.4
2                              Driving (drive alone)         98.5
M-1-a - Low Income Access Pct  -  Mohub
                                                 mode  rounded_val
46                                               Walk         94.2
42                                               Bike         99.9
47                  Walk, Micromobility, Microtransit         94.7
43            Walk, Bike, Micromobility, Microtransit         99.9
45  Transit - Accessed by Walk and or Flexible Fle...         89.8
44                              Driving (drive alone)

In [85]:
df = final_output[final_output['measure'] == 'M-1-c - Mobility Hubs']
df = df[df['population'] == 'Senior Access Pct ']

df[['mode', 'rounded_val']]

,mode,rounded_val
195,Transit - Accessed by Walk and or Flexible Fle...,96.4
194,Driving (drive alone),100.0


In [54]:
final_output

,scenario_id,measure,metric,rounded_val,og_val,updated_by,updated_date,population,mode,output_ranking
94,767,M-1-b,Minority Access Pct - Walk,55.2,0.551621,SANDAGNET\ndh,2022-12-14 12:56:00,Minority Access Pct,Walk,1
70,767,M-1-a - Mobility Hubs,Non-Senior Access Pct - Walk,91.1,0.911066,SANDAGNET\ndh,2022-12-14 12:54:00,Non-Senior Access Pct,Walk,1
22,767,M-1-a,Non-Minority Access Pct - Walk,64.0,0.639954,SANDAGNET\ndh,2022-12-14 12:51:00,Non-Minority Access Pct,Walk,1
118,767,M-1-b,Population Access Pct - Walk,51.0,0.510051,SANDAGNET\ndh,2022-12-14 12:56:00,Population Access Pct,Walk,1
64,767,M-1-a - Mobility Hubs,Non-Minority Access Pct - Walk,91.3,0.913157,SANDAGNET\ndh,2022-12-14 12:54:00,Non-Minority Access Pct,Walk,1
...,...,...,...,...,...,...,...,...,...,...
164,767,M-1-b - Mobility Hubs,Senior Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 12:58:00,Senior Access Pct,Driving (drive alone),6
68,767,M-1-a - Mobility Hubs,Non-Senior Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 12:54:00,Non-Senior Access Pct,Driving (drive alone),6
168,767,M-1-c,Low Income Access Pct - Drive Alone,100.0,0.999950,SANDAGNET\ndh,2022-12-14 13:01:00,Low Income Access Pct,Driving (drive alone),6
56,767,M-1-a - Mobility Hubs,Non-Low Income Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 12:54:00,Non-Low Income Access Pct,Driving (drive alone),6
